In [1]:
obs = ("A", "C", "C", "G", "T", "G", "C", "A")
states = ("H", "L")
start_p = {"H": 0.5, "L": 0.5}
trans_p = {
    "H": {"H": 0.5, "L": 0.5},
    "L": {"H": 0.4, "L": 0.6},
}
emit_p = {
    "H": {"A": 0.2, "C": 0.3, "G": 0.3, "T": 0.2},
    "L": {"A": 0.3, "C": 0.2, "G": 0.2, "T": 0.3},
}

In [11]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    best_st = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print ("The steps of states are " + " ".join(opt) + " with highest probability of %s" % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " " * 5 + "     ".join(("%3d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.15s: " % state + " ".join("%.15s" % ("%.15f" % v[state]["prob"]) for v in V)

In [12]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

       0       1       2       3       4       5       6       7
H: 0.1000000000000 0.0180000000000 0.0027000000000 0.0004050000000 0.0000405000000 0.0000072900000 0.0000010935000 0.0000001093500
L: 0.1500000000000 0.0180000000000 0.0021600000000 0.0002700000000 0.0000607500000 0.0000072900000 0.0000008748000 0.0000001640250
The steps of states are L H H H L H H L with highest probability of 1.6402499999999994e-07
